## Motivation

Practice training a model using AzureML and explore functionalities and capabilities such as provisioning a compute cluster, running experiments

## Scenario

## Strategy



In [15]:
pip install azureml.widgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.8/125.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.7 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.7/245.7 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 48.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.1/439.1 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 99.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 25.9 MB/s eta 0:00:00
    

In [16]:
# Load Libraries

import azureml.core
from azureml.core import Workspace

import os, shutil

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails

from azureml.core import Model


In [2]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))
     

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
Ready to use Azure ML 1.48.0 to work with workbench


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AFKXH8YCZ to authenticate.


In [4]:
# Create a folder for the experiment files
training_folder = 'insurance-training'
os.makedirs(training_folder, exist_ok=True)


In [8]:
# Copy the data file into the experiment folder
shutil.copy('data/insurance.csv', os.path.join(training_folder, "insurance.csv"))

'insurance-training/insurance.csv'

In [9]:
%%writefile environment/environment.yml


# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: simple_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- pandas
- pip
- pip:
  - azureml-defaults
  - azureml-mlflow

Writing environment/environment.yml


In [18]:
%%writefile $training_folder/insurance_training.py

# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Get the experiment run context
run = Run.get_context()

# load the dataset
print("Loading Data...")
df = pd.read_csv('insurance.csv')

# Separate features and labels
X = df[['age','bmi','children']].values
y = df['charges'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and Train the Model
model = LinearRegression().fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# calculate r2
r2 = model.score(X_test, y_test)
run.log('r2', round(r2,2))

# calculate rmse
rmse = mean_squared_error(y_test, y_pred)
run.log('RMSE', np.float(rmse))

# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/insurance_model.pkl')

run.complete()

Overwriting insurance-training/insurance_training.py


In [12]:
## Provision Compute Cluster or use existing

cluster_name = "sweetcluster"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


## Run the training script as an experiment

Using ScriptRunConfig, parameters are gathered to run the experiment. With the env variable, the yaml file essentially contains instructions on how to build these virtualized containers.  These containers will run on the compute cluster defined above.  AzureML will first provision the compute instances of the cluster and then create containers. This will take some time for set-up.  Afterwards, it will shortly execute the training script to generate a model and results. By using environments, we are creating a repeatable process to run code.

In [25]:


# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("experiment_env", "environment/environment.yml")

# Create a script config
script_config = ScriptRunConfig(source_directory= training_folder,
                                script='insurance_training.py',
                                environment=env,
                                docker_runtime_config=DockerConfiguration(use_docker=True),
                                compute_target=cluster_name) 

# submit the experiment run
experiment_name = 'mslearn-train-insurance'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)

# Show the running experiment run in the notebook widget
RunDetails(run).show()

# Block until the experiment run has completed
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'mslearn-train-insurance_1677126848_dfeb6853',
 'target': 'sweetcluster',
 'status': 'Completed',
 'startTimeUtc': '2023-02-23T04:46:49.635066Z',
 'endTimeUtc': '2023-02-23T04:47:59.11169Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': 'c5044332-20c1-4217-8e99-c789d6c0ba77',
  'azureml.git.repository_uri': 'https://github.com/godot107/azureml.git',
  'mlflow.source.git.repoURL': 'https://github.com/godot107/azureml.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': '549c6237ee416ef2dd7ebc4a4661c414fd5a164b',
  'mlflow.source.git.commit': '549c6237ee416ef2dd7ebc4a4661c414fd5a164b',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'insurance_training.py',
  'command': '',
  'useAbsolutePath': False,
  'argume

Unexpected exception formatting exception. Falling back to standard exception


In [26]:

# Get logged metrics and files
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

r2 0.15
RMSE 131201335.64669803


azureml-logs/20_image_build_log.txt
outputs/insurance_model.pkl
system_logs/cs_capability/cs-capability.log
system_logs/hosttools_capability/hosttools-capability.log
system_logs/lifecycler/execution-wrapper.log
system_logs/lifecycler/lifecycler.log
system_logs/metrics_capability/metrics-capability.log
system_logs/snapshot_capability/snapshot-capability.log
user_logs/std_log.txt


## Register the Model

With the model artifact created from the training script, we can save the model within our Azure Workspace.  

In [31]:
# Register the model
run.register_model(model_path='outputs/insurance_model.pkl', model_name='insurance_model',
                   tags={'Training context':'Script'},
                   properties={'r2': run.get_metrics()['r2'], 'RMSE': run.get_metrics()['RMSE']})

Model(workspace=Workspace.create(name='workbench', subscription_id='71fa0172-ce90-403c-94a9-14ce1e88f56a', resource_group='rg_eastus_44930_1_16771245691802'), name=insurance_model, id=insurance_model:2, version=2, tags={'Training context': 'Script'}, properties={'r2': '0.15', 'RMSE': '131201335.64669803'})

After creating the model, we can decide to deploy as an endpoint to make inferences.

In [32]:
# Fetches latest model
model = ws.models['insurance_model']
print(model.name, 'version', model.version)

insurance_model version 2


In [33]:
import os

# Create a folder for the deployment files
deployment_folder = './insurance_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_insurance.py'
script_path = os.path.join(deployment_folder,script_file)

./insurance_service folder created.


In [34]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'insurance_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)

    return json.dumps(predictions)

Writing ./insurance_service/score_insurance.py


In [36]:
from azureml.core import Environment
from azureml.core import Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Configure the scoring environment
service_env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference")
service_env.inferencing_stack_version="latest"

inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=service_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model as a service
print('Deploying model...')
service_name = "insurance-service"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)

/tmp/ipykernel_11836/139018624.py:20: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
/tmp/ipykernel_11836/3165749768.py:20: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-onl

In [37]:

for webservice_name in ws.webservices:
    print(webservice_name)

insurance-service
diabetes-service


In [40]:
## Make Inferences

import json

x_new = [[31,25.74,0]]

input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_charges= json.loads(predictions)

print(predicted_charges)

Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '47', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Thu, 23 Feb 2023 05:13:24 GMT', 'Server': 'nginx', 'X-Ms-Client-Request-Id': '4361dc9d-38f9-420f-b037-3b3c9f03c216', 'X-Ms-Request-Id': '4361dc9d-38f9-420f-b037-3b3c9f03c216', 'X-Ms-Run-Function-Failed': 'True', 'X-Ms-Server-Version': 'azmlinfsrv/0.7.6', 'X-Request-Id': 'a71213dd-fb63-484e-9c10-1066856d7c2f'}
Content: b'Object of type ndarray is not JSON serializable'



WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '47', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Thu, 23 Feb 2023 05:13:24 GMT', 'Server': 'nginx', 'X-Ms-Client-Request-Id': '4361dc9d-38f9-420f-b037-3b3c9f03c216', 'X-Ms-Request-Id': '4361dc9d-38f9-420f-b037-3b3c9f03c216', 'X-Ms-Run-Function-Failed': 'True', 'X-Ms-Server-Version': 'azmlinfsrv/0.7.6', 'X-Request-Id': 'a71213dd-fb63-484e-9c10-1066856d7c2f'}
Content: b'Object of type ndarray is not JSON serializable'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.\nResponse Code: 502\nHeaders: {'Connection': 'keep-alive', 'Content-Length': '47', 'Content-Type': 'text/html; charset=utf-8', 'Date': 

Research/References:

- https://github.com/MicrosoftLearning/mslearn-dp100/blob/main/05%20-%20Train%20Models.ipynb


Looking Ahead:
- 